In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
import re
import scipy
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop = stopwords.words('english')
from sklearn.feature_extraction.text import TfidfTransformer
#Helper Functions 




def utils_preprocess_text(text):
    
    #text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'[^[a-zA-Z]\s]', '', str(text).lower().strip())
  
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    if stop is not None:
      lst_text = [word for word in lst_text if word not in 
                  stop]
                

    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text


def fasterRemoveStopWords(list_of_words):
    stopwords_set = set(stopwords.words())
    return [word for word in list_of_words if word not in stopwords_set]

def getDataframe():
  path = 'D:/Movie recommendation/MovieSummaries/CombineDataSetLargeGenreIntegrated.csv'
  print("PATH = ",path)
  df = pd.read_csv(path, sep=',',header = None)
  df = df.drop(df.columns[0], axis=1)
  df = df.drop(0, axis=0)
  df.columns = ['MovieName', 'Language', 'Country', 'Genre', 'Summaries', 'text_clean','word_count','text_clean_genre']
  df.reset_index()  
  return df

def gettfidf():
  with open('D:/Movie recommendation/MovieSummaries/tfidfmodel.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
  return vectorizer

def gettfidf_gen():
  with open('D:/Movie recommendation/tfidf_gen.pkl', 'rb') as f:
    vectorizer_gen = pickle.load(f)
  return vectorizer_gen

def getTfidf_matrix():
  tfidf_matrix_array = np.load('D:/Movie recommendation/MovieSummaries/tfidf_matrix_array.npy')
  tfidf_matrix = scipy.sparse.csr_matrix(tfidf_matrix_array)
  return tfidf_matrix

def getTfidf_matrix_gen():
  tfidf_matrix_array = np.load('D:/Movie recommendation/tfidf_matrix_array_gen.npy')
  tfidf_matrix_gen = scipy.sparse.csr_matrix(tfidf_matrix_array)
  return tfidf_matrix_gen
   


tf = gettfidf()
tfidf_matrix = getTfidf_matrix()
tf_gen=gettfidf_gen()
tfidf_matrix_gen=getTfidf_matrix_gen()
df = getDataframe()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HARSHIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HARSHIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HARSHIT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
c:\Users\HARSHIT\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HARSHIT\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from 

PATH =  D:/Movie recommendation/MovieSummaries/CombineDataSetLargeGenreIntegrated.csv


In [10]:
def get_recommendations_based_on_plot(plot,tf,tfidf_matrix,df):
  titles = df['MovieName']
  plot = utils_preprocess_text(plot)
  tf_vector_plot = tf.transform([plot])
  cosine_sim_plot = linear_kernel(tfidf_matrix, tf_vector_plot)
  sim_scores = list(enumerate(cosine_sim_plot))
  print("sim_scores = ", sim_scores)
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[0:31]
  print("SimScores = ", sim_scores)

  movie_indices = [i[0] for i in sim_scores]
  return df['MovieName'].iloc[movie_indices]
  #movie_names = [df['MovieName'].iloc[index] for index in movie_indices]
  #return titles.iloc[movie_indices]
  #return movie_names

#text = "The movie is about a pilot Maveric who flies fighter planes with his friend. They go to a school called Top Gun to become better pilots"
#text = "A Science Fiction Movies related to aliens and humans exploring different planets and using advanced technology"
#recc = get_recommendations_based_on_plot(text)[:5]

#print("RESPONSE = ", recc)



In [11]:
import time
import numpy as np
from flask import Flask, request, make_response
import json
import pickle

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))
@app.route('/')
def hello():
    return 'Hello World. I am finally connected to the web.'

# geting and sending response to dialogflow
@app.route('/webhook', methods=['POST'])
def webhook():
    if request.method == 'POST':
        print('Webhook Received')


    req = request.get_json(silent=True, force=True)
    print(req)
    res = processRequest(req)
    res = json.dumps(res)

    r = make_response(res)
    r.headers['Content-Type'] = 'application/json'
    return r  #Final Response sent to DialogFlow

def processRequest(req):    # This method processes the incoming request 
    print("came into process function")
    result = req.get("queryResult")
    querytext=result.get("queryText")
    print(querytext)
    parameters = result.get("parameters")
    genre=parameters.get("genre")
    intent = result.get("intent").get('displayName')
    print(intent)
    if (intent=='plot_file'):
        print("StartTime = ",time.time())
        rec_movie=get_recommendations_based_on_plot(querytext,tf,tfidf_matrix,df)[:1]
        print("StartTime = ",time.time())
        index=0 
        value='default'
        for index_1, value_1 in rec_movie.iteritems():
            index=index_1
            value=value_1
        tf_x_test=tf_gen.transform([querytext])
        print(tf_x_test)
        prediction = model.predict(tf_x_test)
        print(type(prediction))
        output = str(prediction)
        print(type(output))
        if output=='[1]':
            output='Comedy'
        elif output=='[2]':
            output='Drama'
        elif output=='[3]':
            output= 'Romance Film'
        elif output =='[4]':
            output = 'Thriller'
        else:
            output='Action'
        return {
            "fulfillmentText": "The recommeded movie is: {} with genre {} !".format(value,output)
        }

if __name__ == '__main__':
    #nltk.download('stopwords')
    #nltk.download('wordnet')
    #nltk.download('omw-1.4')
    #stop = stopwords.words('english')
    #tf = gettfidf()
    #tfidf_matrix = getTfidf_matrix()
    #df = getDataframe()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Webhook Received
{'responseId': '85479476-8527-4f9a-a277-61559241365f-bfbd87a2', 'queryResult': {'queryText': 'Movies related to aliens and humans exploring different planets and using advanced technology', 'parameters': {}, 'allRequiredParamsPresent': True, 'fulfillmentMessages': [{'text': {'text': ['']}}], 'outputContexts': [{'name': 'projects/movie-recommendation-fina-jkkn/agent/sessions/d71ed0b0-2ae5-7933-421f-9a2fdea286e6/contexts/defaultwelcomeintent-followup', 'parameters': {'genre': '', 'genre.original': ''}}, {'name': 'projects/movie-recommendation-fina-jkkn/agent/sessions/d71ed0b0-2ae5-7933-421f-9a2fdea286e6/contexts/plot_out', 'lifespanCount': 4, 'parameters': {'genre': '', 'genre.original': ''}}, {'name': 'projects/movie-recommendation-fina-jkkn/agent/sessions/d71ed0b0-2ae5-7933-421f-9a2fdea286e6/contexts/__system_counters__', 'parameters': {'no-input': 0.0, 'no-match': 0.0}}], 'intent': {'name': 'projects/movie-recommendation-fina-jkkn/agent/intents/7978c551-cf82-4d7a-bc7d

127.0.0.1 - - [06/Apr/2023 14:26:29] "POST /webhook HTTP/1.1" 200 -


[(0, array([0.02586909])), (1, array([0.])), (2, array([0.])), (3, array([0.])), (4, array([0.04908081])), (5, array([0.])), (6, array([0.01107556])), (7, array([0.00868551])), (8, array([0.])), (9, array([0.05226855])), (10, array([0.])), (11, array([0.00701245])), (12, array([0.00646466])), (13, array([0.01152045])), (14, array([0.0071122])), (15, array([0.])), (16, array([0.00232013])), (17, array([0.])), (18, array([0.])), (19, array([0.])), (20, array([0.00759261])), (21, array([0.])), (22, array([0.00581591])), (23, array([0.03080229])), (24, array([0.])), (25, array([0.00241369])), (26, array([0.00515892])), (27, array([0.00312795])), (28, array([0.04338442])), (29, array([0.01170712])), (30, array([0.00903914])), (31, array([0.00618166])), (32, array([0.1486838])), (33, array([0.])), (34, array([0.00956297])), (35, array([0.10951503])), (36, array([0.02722198])), (37, array([0.00289387])), (38, array([0.03752701])), (39, array([0.])), (40, array([0.0366557])), (41, array([0.]))

127.0.0.1 - - [06/Apr/2023 14:27:18] "POST /webhook HTTP/1.1" 200 -


[(0, array([0.])), (1, array([0.])), (2, array([0.])), (3, array([0.])), (4, array([0.03748259])), (5, array([0.])), (6, array([0.])), (7, array([0.00993191])), (8, array([0.])), (9, array([0.08458651])), (10, array([0.])), (11, array([0.00693014])), (12, array([0.])), (13, array([0.])), (14, array([0.])), (15, array([0.])), (16, array([0.])), (17, array([0.])), (18, array([0.])), (19, array([0.])), (20, array([0.00210547])), (21, array([0.])), (22, array([0.])), (23, array([0.0306663])), (24, array([0.])), (25, array([0.])), (26, array([0.])), (27, array([0.])), (28, array([0.07020926])), (29, array([0.])), (30, array([0.])), (31, array([0.0061091])), (32, array([0.22616834])), (33, array([0.])), (34, array([0.])), (35, array([0.1665873])), (36, array([0.02922637])), (37, array([0.00468317])), (38, array([0.06073019])), (39, array([0.])), (40, array([0.])), (41, array([0.])), (42, array([0.00735309])), (43, array([0.00461538])), (44, array([0.])), (45, array([0.])), (46, array([0.0084

127.0.0.1 - - [06/Apr/2023 14:28:13] "POST /webhook HTTP/1.1" 200 -


[(0, array([0.02678926])), (1, array([0.])), (2, array([0.])), (3, array([0.])), (4, array([0.])), (5, array([0.])), (6, array([0.])), (7, array([0.])), (8, array([0.])), (9, array([0.06386064])), (10, array([0.])), (11, array([0.01192879])), (12, array([0.])), (13, array([0.])), (14, array([0.])), (15, array([0.])), (16, array([0.])), (17, array([0.])), (18, array([0.])), (19, array([0.])), (20, array([0.00362413])), (21, array([0.])), (22, array([0.])), (23, array([0.01296687])), (24, array([0.])), (25, array([0.])), (26, array([0.])), (27, array([0.])), (28, array([0.])), (29, array([0.])), (30, array([0.])), (31, array([0.01051554])), (32, array([0.10327348])), (33, array([0.])), (34, array([0.])), (35, array([0.])), (36, array([0.])), (37, array([0.00806109])), (38, array([0.])), (39, array([0.02579158])), (40, array([0.])), (41, array([0.])), (42, array([0.0126568])), (43, array([0.00794441])), (44, array([0.])), (45, array([0.])), (46, array([0.01461319])), (47, array([0.])), (4